In [ ]:
from datasets import Dataset

import sys
sys.path.append("..")

from examples.prepare_infoseek_kb import *
from unimemrag.utils.data_loader import *


kb_path = "../benchmark/infoseek/wiki_text/infoseek_kb_100k.jsonl"

dataset = load_infoseek_dataset(
    kb_path,
    prefer_summary=True,          # optional: use summaries when available
    filter_missing_images=False,  # optional: drop rows without local images
)

In [ ]:
print(dataset[10])

In [ ]:
print(len(dataset))

In [ ]:
from unimemrag.main import (
    RagPipelineConfig,
    create_retriever,
    load_viquae_dataset,
    index_text_corpus,
    index_image_corpus,
    build_rag_bundle,
    evaluate_bundle_with_model,
)

cfg = RagPipelineConfig(
    collection="infoseek",
    dataset_file="benchmark/infoseek/wiki_text/infoseek_kb_100k.jsonl",
    image_root="benchmark/infoseek/images_100k",
    top_k=5,
)


In [ ]:
retriever, embed_model, vector_store = create_retriever(cfg)

In [ ]:
index_text_corpus(dataset, embedder=embed_model, store=vector_store, batch_size=cfg.text_batch_size)
# index_image_corpus(dataset, embedder=embed_model, store=vector_store, image_root=Path(cfg.image_root))

In [ ]:
image_paths = sorted(Path("../benchmark/infoseek/images_100k").glob("*"))
result = vector_store.ingest_images(
    embed_model,
    image_paths,
    caption_builder=None,
    show_progress=True,
)

In [ ]:
print(result["images_indexed"], "images indexed")

In [ ]:
# vector_store.clear_collection()

In [1]:
import sys
sys.path.append('..')

import os
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

from config import Config
from unimemrag.main import RagPipelineConfig
from unimemrag.embedding.models.ClipEmbedding import ClipEmbedding
from unimemrag.vector_store.qdrant import QdrantStore
from unimemrag.retriever.retriever import Retriever

cfg = RagPipelineConfig(
    collection="infoseek",
    dataset_file="benchmark/infoseek/wiki_text/infoseek_kb_100k.jsonl",
    image_root="benchmark/infoseek/images_100k"
)
embedder = ClipEmbedding(cfg.model_name)
store = QdrantStore(cfg, vector_size=embedder.dim) 
retriever = Retriever(embedder, store, top_k=10, image_top_k=3)

/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/examples/../unimemrag/vector_store/qdrant.py:40: UserWarning: Qdrant client version 1.15.1 is incompatib

In [3]:
question = "The image you've provided shows a telescope, which is an optical instrument used to view objects that are too distant or small to be seen with the naked eye. Telescopes can be created by various manufacturers and individuals, depending on their purpose and design."

nodes = retriever.search_by_image(
    image="../../benchmark/oven_eval/image_downloads/oven_images/04/oven_04994979.JPEG",
    # target_modality="text",
    as_nodes=True,
)

In [4]:
nodes

[ImageNode(id='7f24afbe-308f-50b0-b04c-906a3996371f', score=0.84898823, uri='/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q6126018.jpg', content='Q6126018', modality='image', payload={'uri': '/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q6126018.jpg', 'modality': 'image', 'content': 'Q6126018'}),
 ImageNode(id='f3d60d1c-e732-520e-9309-7b75f61643ba', score=0.74023926, uri='/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q709529.gif', content='Q709529', modality='image', payload={'uri': '/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q709529.gif', 'modality': 'image', 'content': 'Q709529'}),
 ImageNode(id='3ecd9154-d8c0-52cb-9166-f529d3ebac5d', score=0.7020745, uri='/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q6155807.jpg', content='Q6155807', modality='image', payload={'uri': '/hpc2hdd/home/sgu

In [ ]:
from unimemrag.vlm.QwenVL import QwenVL

vlm = QwenVL(
      model_path="../../ckpts/Qwen3-VL-8B-Instruct",
      torch_dtype="auto",
      device_map="auto",
      # device_map={"": "cuda:0"}
      attn_implementation="flash_attention_2",
      # min_pixels=256*28*28, max_pixels=1280*28*28,  # 如需控制视觉Token范围可解开
)

In [ ]:
from pathlib import Path

question = "who created this object?"
image_path = "../benchmark/oven_eval/image_downloads/oven_images/04/oven_04994979.JPEG"


answer, used_nodes = retriever.answer_with_vlm(
    query=question,
    vlm=vlm,
    nodes=nodes,
    gen_kwargs={"temperature": 0.7, "max_new_tokens": 4096},
)

print(answer)

2025-11-02 18:17:46 | INFO     | retriever:_log_vlm_input:109 - VLM input prepared | meta={'query': 'What country does this place belong to?', 'note': 'vlm_with_retrieved_context', 'node_count': 13, 'nodes': [{'id': '7f24afbe-308f-50b0-b04c-906a3996371f', 'uri': '/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q6126018.jpg', 'modality': 'image', 'content': 'Q6126018', 'score': 0.84898823}, {'id': 'f3d60d1c-e732-520e-9309-7b75f61643ba', 'uri': '/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q709529.gif', 'modality': 'image', 'content': 'Q709529', 'score': 0.74023926}, {'id': '3ecd9154-d8c0-52cb-9166-f529d3ebac5d', 'uri': '/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/benchmark/infoseek/images_100k/Q6155807.jpg', 'modality': 'image', 'content': 'Q6155807', 'score': 0.7020745}, {'id': 'd4ba52c1-59c6-5129-ae79-b39922af2cbf', 'uri': 'Q5446999', 'modality': 'text', 'content': 'Field flattener lens is a type of

In [11]:
messages = [
      {
        "role": "system",
        "content": "You are a helpful assistant.",
      },
      {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": "What country does this place belong to?"},
        ],
    }
    ]
out = vlm.chat(messages, max_new_tokens=4096)
print("chat:", out)

FileNotFoundError: [Errno 2] No such file or directory: '../benchmark/oven_eval/image_downloads/oven_images/04/oven_04951981.JPEG'